# Co-occurrence Analysis

Cognitive Control has many faces and for this reason a good part of tests literature targets the Cognitive Control using multiple tests in a single study. On that note, the co-occurrences of tests in the literature help us identify tests that are commonly used together, tests that are validated by other tests, and tests that can be ignored due to redundancy or lack of correlation with other standard tests. This co-occurence analysis improves our understanding of the coverage and dependencies in the Cognitive Control tests.

This notebooks demonstrates analyses of co-occurences of cognitive tests in PubMed articles.

- [ ] rename variables in a consistent way

In [3]:
# Setup

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='white')
from pathlib import Path
import pandas as pd
import numpy as np
import networkx as nx

pubmed_fpath = Path('../data/pubmed/tests/')
_corpora = []

# fetch csv file names and remove irrelevant tasks
files = [f for f in pubmed_fpath.glob('*.csv') if f.stem not in ['STOP', 'MONITOR']]

for f in files:
    _df = pd.read_csv(f)
    _df['cognitive_test'] = f.stem
    _corpora.append(_df)

CORPUS = pd.concat(_corpora)


cognitive_tests = CORPUS['cognitive_test'].unique().tolist()

First we need to calculate co-ocurence of two tests given the two corresponding corpora. This co-occurence feaure is conditional (i.e., bidirectional) given which of the two corpora is being analyzed.

The following code create a matrix called `cooccurences` of size n*n where n is the total number of tests and its element `[i,j]` contains the number of articles in corpus `j` that also exist in corpus `i`. Note that the matrix is symmetrix and `x[i,j] == x[j,i]`.

A second matrix which is called `cooccurences_normalized_pct` contains the same information but values in `[i,j]` are percetage of corpus `j` that also exist in the corpus `i`; values are normalized by the number of total articles in corpus `j`. Normalized co-occurence matrix is not symmetric.

Here is a list of all produced matrices (we don't need all):

- `cooccurences`
- `cooccurences_normalized`
- `cooccurences_normalized_pct`
- `undirected_cooccurences_normalized`
- `undirected_cooccurences_normalized_pct`


- [ ] **TODO:** Is there a better way to explain how to interpret the cooccurences matrix? Indexing somehow confuses me.

In [3]:
cooccurences = np.zeros((len(cognitive_tests), len(cognitive_tests)))
cooccurences_normalized = np.zeros_like(cooccurences)
undirected_cooccurences_normalized = np.ones_like(cooccurences)

def find_union_set(left_set, right_set):
    """Find shared items (union set) in two sets of PMIDs."""
    union_set = [_id for _id in left_set if _id in right_set]
    # or set.union(...)
    return union_set

# now fill the co-occurence matrices
for i, test_i in enumerate(cognitive_tests):
    pmids_i = set(CORPUS.query('cognitive_test == @test_i')['pmid'])
    for j, test_j in enumerate(cognitive_tests):
        if i!=j:
            pmids_j = set(CORPUS.query('cognitive_test == @test_j')['pmid'])
            cooccurences[i,j] = len(find_union_set(pmids_i, pmids_j))
            cooccurences_normalized[i,j] = cooccurences[i,j] / len(pmids_j)
            undirected_cooccurences_normalized[i,j] = 2 * cooccurences[i,j] / (len(pmids_i) + len(pmids_j))

cooccurences_normalized_pct = cooccurences_normalized * 100
undirected_cooccurences_normalized_pct = undirected_cooccurences_normalized * 100

# DEBUG cooccurences, cooccurences_normalized

Let's look at the result by finding the most co-occured cognitive tests:

In [4]:
max_index = np.unravel_index(np.argmax(cooccurences, axis=None), cooccurences.shape)
max_norm_index = np.unravel_index(np.argmax(cooccurences_normalized_pct, axis=None), cooccurences_normalized_pct.shape)

print(f'max co-ocurence: "{cognitive_tests[max_index[0]]}" in "{cognitive_tests[max_index[1]]}" corpus')
print(f'max co-ocurence (normalized by corpus size): "{cognitive_tests[max_norm_index[0]]}" and "{cognitive_tests[max_norm_index[1]]}"')

max co-ocurence: "Stroop" in "TMT (Trail Making Test)" corpus
max co-ocurence (normalized by corpus size): "Sorting Task" and "Spin the Pots"


Let's plot a simple heatmap in that each cell shows co-occurence of test pairs.

In [12]:
_, ax = plt.subplots(1,1,figsize=(25,25))
sns.heatmap(cooccurences,
            xticklabels=cognitive_tests,
            yticklabels=cognitive_tests,
            cbar_kws = {'use_gridspec':False, 'location':'top', 'shrink': .5},
            square=True,
            linewidths=.5,
            linecolor='k',
            ax=ax)

plt.xticks(rotation=45, ha='right') 
ax.set(title='Bidirectional co-occurence of cognitive tests\n Values are the number of articles shared between row corpus and col corpus')
plt.savefig('../outputs/cooccurences/heatmap.png')
plt.close()

# Now, same plot for the normalized co-occurences matrix

_, ax = plt.subplots(1,1,figsize=(25,25))
sns.heatmap(cooccurences_normalized_pct,
            xticklabels=cognitive_tests,
            yticklabels=cognitive_tests,
            cbar_kws = {'use_gridspec':False, 'location':'top', 'shrink': .5},
            square=True,
            linewidths=.5,
            linecolor='k',
            ax=ax)

plt.xticks(rotation=45, ha='right') 
ax.set(title='Bidirectional normalized co-occurence of cognitive tests \n (row=i, col=j, values=% of coprus j)')
plt.savefig('../outputs/cooccurences/heatmap_normalized.png')
plt.close()

Here is the same heatmap, but tests are clustered:

In [5]:
# plot cluster map for co-occurence counts
sns.clustermap(cooccurences,
            xticklabels=cognitive_tests,
            yticklabels=cognitive_tests,
            figsize=(15,15))

plt.xticks(rotation=45, ha='right') 
plt.suptitle('Bidirectional co-occurence of cognitive tests')
plt.savefig('../outputs/cooccurences/clustermap.png')
plt.close()

# plot cluster map for normalized co-occurence
sns.clustermap(cooccurences_normalized_pct,
            xticklabels=cognitive_tests,
            yticklabels=cognitive_tests,
            figsize=(15,15))

plt.xticks(rotation=45, ha='right') 
plt.suptitle('Bidirectional normalized co-occurence of cognitive tests\n(row=i, col=j, value=pct of corpus j)')
plt.savefig('../outputs/cooccurences/clustermap_normalized.png')
plt.close()

/Users/morteza/miniconda3/lib/python3.8/site-packages/seaborn/matrix.py:629: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage = hierarchy.linkage(self.array, method=self.method,


## Visualization preprocessing 

Before plotting co-occurence map as a graph, some proprocessing steps significantly improve the visuals:

1. remove low degree nodes and low weight edges.
2. Use degree centrality as node sizes; degree centrality shows number of connected edges to a given nodes (both inputs and outputs). Another measure that can be used for node sizes is the number of non-co-occured articles that only mentioned once in the related test coprus.
3. Find communities in the graph, that are highly connected subgraphs. This property can be used to layout the graph and colorize the nodes.
4. Put co-occurred tests closer to each other by using force-driven spring layout.

I will use the de-facto NetworkX package to perform all the graph analysis listed above. The output will be a bidirectional graph that encapsulates centrality of each node, communities, edge widths, and positioning layout.

- [ ] **TODO:** greedy modulatiry algorithm returns one community because nodes are densly connected. Use another algorithm to extract some information from the graph structure.

In [9]:
# normalized co-occurences below this threshold will be ignored
MIN_EDGE_WEIGHT = 5 # percent

# create graph from adjacency matrix
G = nx.from_numpy_matrix(cooccurences_normalized_pct, create_using=nx.DiGraph)

# set node labels
node_labels = {i:t for i,t in enumerate(cognitive_tests)}
nx.set_node_attributes(G, node_labels, 'label')

# remove low weight edges and low degree nodes
low_weight_edges = [(u,v) for u, v in G.edges() if G[u][v]['weight'] < MIN_EDGE_WEIGHT]
G.remove_edges_from(low_weight_edges)

low_degree_nodes = [n for n, d in G.degree() if d == 0]
G.remove_nodes_from(low_degree_nodes)


print(f'[INFO] After removing {len(low_weight_edges)} low-weight edges, {len(G.edges())} remained.')
print(f'[INFO] The following low-degree nodes are removed:\n', '\n '.join([cognitive_tests[ldn] for ldn in low_degree_nodes]))


# Community detection

def heaviest(G):
    from operator import itemgetter
    u, v, w = max(G.edges(data='weight'), key=itemgetter(2))
    return (u,v)

import itertools


k = 2   # number of communities
communities = list(
        itertools.takewhile(lambda c: len(c) <= k, nx.community.girvan_newman(G, most_valuable_edge=heaviest))
    )[-1]

# this does not work for DiGraph
# communities = nx.community.greedy_modularity_communities(G,weight='weight')

community_index = {n: i for i, comm in enumerate(communities) for n in comm}
nx.set_node_attributes(G, community_index, 'community')


# compute centrality
centrality = nx.degree_centrality(G)
nx.set_node_attributes(G, centrality, 'centrality')

# visual properties (all numeric values are relative, update them to match your canvas)
node_colors = community_index
edge_widths = [G[u][v]['weight'] for u, v in G.edges()]


# compute layout positions
pos = nx.spring_layout(G, k=.8)

[INFO] After removing 912 low-weight edges, 228 remained.
[INFO] The following low-degree nodes are removed:
 Balance Beam Task
 Auditory Attention
 RNG (Random Number Generation Task)
 Incompatibility Test
 Odd One Out
 Self Control Schedule
 PVT (Psychomotor Vigilance Task)
 D2 Target Detection
 Object Substitution Task
 PEG (Pencil Tapping Task)
 Category Switch Task
 Analogy Making Task
 Delayed Alteration Task
 Dimension Switching Task
 Nine Boxes


# NetworkX plot (static)

In [118]:
fig, ax = plt.subplots(1,1,figsize=(15,15))

nx.draw_networkx_nodes(G, pos,
                       node_color=list(node_colors.values()),
                       node_size=[max(10,v*10000) for v in centrality.values()],
                       alpha=0.5, ax=ax)
nx.draw_networkx_labels(G, pos, labels=nx.get_node_attributes(G, 'label'), font_size=8, alpha=0.5, ax=ax)
# nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.3, connectionstyle='arc3,rad=0.1')\n",
nx.draw_networkx_edges(G,
                       pos,
                       alpha=0.8,
                       arrowstyle='->',
                       arrowsize=10,
                       connectionstyle='arc3,rad=0.1',
                       width=[v/10 for v in  edge_widths],
                       edge_color='gainsboro',
                       ax=ax)

# plot the graph using matplotlib backend
plt.suptitle('Co-occurence map\n Node size and edge width respectively show centrality degree and number of co-occured articles in ith corpus.')
plt.tight_layout()
plt.savefig('../outputs/cooccurences/map_networkx.png')
plt.close()

## Bokeh plot (interactive)

In [127]:

from bokeh.io import output_notebook, show; output_notebook()
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,)
from bokeh.plotting import from_networkx

plot = Plot(x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))

graph_renderer = from_networkx(G, nx.spring_layout(G, k=5), scale=1, center=(0,0))

# data
graph_renderer.node_renderer.data_source.data['node_size'] = [max(10,v*100) for v in centrality.values()]
graph_renderer.edge_renderer.data_source.data['edge_size'] = [max(.1, v/5) for v in edge_widths]

# nodes
graph_renderer.node_renderer.glyph = Circle(size='node_size', fill_color='lightblue', line_color='white')
graph_renderer.node_renderer.selection_glyph = Circle(size='node_size', fill_color='mediumpurple', line_color='mediumpurple')
graph_renderer.node_renderer.nonselection_glyph = Circle(size='node_size', fill_color='lightblue', line_color='white')
graph_renderer.node_renderer.hover_glyph = Circle(size='node_size', fill_color='mediumpurple', line_color='mediumpurple')

# edges
graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width='edge_size')
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color='mediumpurple', line_width='edge_size')
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color='mediumpurple', line_width='edge_size')

# tools
node_hover_tool = HoverTool(tooltips=[('Label','@label')])
plot.add_tools(node_hover_tool, TapTool())
graph_renderer.selection_policy = NodesAndLinkedEdges()
# graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

plot.title.text = "Interactive co-occurence map of cognitive tests"
show(plot)

Loading BokehJS ...

## PyVis plot (interactive)

- [ ] **TODO:** bidirectional edges should be separated.

In [131]:
from pyvis.network import Network

nt = Network(height='1000px', width='1000px', directed=False, notebook=True, heading='Cognitive tests co-occurence map')

nt.from_nx(G)

for i,n in enumerate(nt.nodes):
    n['size'] = max(10, n['centrality'] * 100)
    n['color'] = n['community']

for e in nt.edges:
    e['value'] = e['weight']



pyvis_options = """
var options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": 0,
      "centralGravity": 0,
      "springLength": 400,
      "springConstant": 0.1,
      "damping": 1,
      "avoidOverlap": 1
    }
  },
  "nodes": {
    "color": {
      "border": "white",
      "background": "lightblue",
      "highlight": {
        "border": "mediumpurple",
        "background": "mediumpurple"
      }
    }
  },
  "edges": {
    "arrows": {
      "to": {
        "enabled": true
      },
      "from": {
        "enabled": true
      }
    },
    "arrowStrikethrough": false,
    "color": {
      "color": "#dcdcdc88",
      "highlight": "mediumpurple",
      "inherit": false
    },
    "smooth": {
      "type": "continuous",
      "forceDirection": "none"
    }
  }
}
"""

nt.set_options(pyvis_options)
# nt.show_buttons(filter_=['nodes'])

nt.show('../outputs/cooccurences/map_pyvis.html')

### Plotly Sankey (interactive)

There are two sankey plots, first showing number of shared articles, and then percent of shared articles (normalized by target corpus).

In [11]:
import plotly.graph_objects as go

# uncomment to open fig in the browser instead of embedding into the notebook
# import plotly.io as pio; pio.renderers.default = 'browser'

MIN_COOCCURENCE = 250

# prep data for sankey
cooc_df = pd.DataFrame(cooccurences).unstack().reset_index().rename(
  columns={'level_0': 'target', 'level_1': 'source', 0: 'cooccurence'}
)

index_to_test_mapping = {i:c for i,c in enumerate(cognitive_tests)}
# df.replace({'task_i': index_to_test_mapping, 'task_j': index_to_test_mapping}, inplace=True)

# only pairs that co-occured at least 50 times
cooc_df = cooc_df.query('cooccurence > @MIN_COOCCURENCE')


fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 10,
      thickness = 10,
      line = dict(color = 'black', width = 0.5),
      label = cognitive_tests,
      color = 'lightblue'
    ),
    link = dict(
      source = cooc_df['source'],
      target = cooc_df['target'],
      value = cooc_df['cooccurence']
  ))])

fig.update_layout(title_text=f'Co-occurence sankey map (only pairs that co-occured in at least {MIN_COOCCURENCE} articles of target corpus)', font_size=10)

fig.write_html('../outputs/cooccurences/sankey_map_plotly.html')

fig.show()


Or plot the normalized percentages:

In [23]:
import plotly.graph_objects as go

# uncomment to open fig in the browser instead of embedding into the notebook
# import plotly.io as pio; pio.renderers.default = 'browser'

MIN_COOCCURENCE_PCT = 20

# prep data for sankey
cooc_df = pd.DataFrame(cooccurences_normalized_pct).unstack().reset_index().rename(
  columns={'level_0': 'target', 'level_1': 'source', 0: 'cooccurence_pct'}
)

index_to_test_mapping = {i:c for i,c in enumerate(cognitive_tests)}
# df.replace({'task_i': index_to_test_mapping, 'task_j': index_to_test_mapping}, inplace=True)

# only pairs that co-occured at least 50 times
cooc_df = cooc_df.query('cooccurence_pct > @MIN_COOCCURENCE_PCT')


fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 10,
      thickness = 10,
      line = dict(color = 'black', width = 0.5),
      label = cognitive_tests,
      color = 'lightblue'
    ),
    link = dict(
      source = cooc_df['source'],
      target = cooc_df['target'],
      value = cooc_df['cooccurence_pct']
  ))])

fig.update_layout(title_text=f'Co-occurence sankey map (only pairs that co-occured in at least {MIN_COOCCURENCE_PCT}% of the target corpus)', font_size=10)

fig.write_html('../outputs/cooccurences/sankey_map_normalized_plotly.html')

fig.show()
